In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model

Base = automap_base()

# reflect the tables

Base.prepare(engine, reflect = True)

In [ ]:
# View all of the classes that automap found

Base.classes.keys()

In [ ]:
# Save references to each table

measurement = Base.classes.measurement

station = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.

recent_date= session.query(measurement.date).order_by(measurement.date.desc()).first()
recent_date

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database.

recent_date = dt.date(2017,8, 23)

# Calculate the date one year from the last date in data set.

last_year =  recent_date - dt.timedelta(days = 365)


# Perform a query to retrieve the data and precipitation scores

query = session.query(measurement.date, measurement.prcp).filter(measurement.date >= last_year).all()


# Save the query results as a Pandas DataFrame and set the index to the date column

measure_df = pd.DataFrame(query)
measure_df.rename(columns = {0: 'date', 1: 'precipitation'}, inplace=True)

measure_df['date']= pd.to_datetime(measure_df['date'], format = '%Y-%m-%d')

measure_df = measure_df.set_index('date')


# Sort the dataframe by date

measure_df = measure_df.sort_values(by = ["date"])
measure_df

# Use Pandas Plotting with Matplotlib to plot the data

measure_df.plot(title = 'Precipitation For 08/2016 - 08/2017' , figsize = (12,8), rot = 45)
plt.legend(loc = 'upper right')
plt.ylim(0,7.7)
plt.ylabel('Inches')
plt.xlabel('Date')


In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data

measure_df.describe()
measure_df

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset

x = len(session.query(station).all())
print(f'Number of Stations: {x}')

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.

ordered_stations = session.query(measurement.station,
                    func.count(measurement.id)).group_by(measurement.station).order_by(func.count(measurement.id).desc()).all()
ordered_stations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
stat = [measurement.station, 
      func.min(measurement.tobs),
      func.max(measurement.tobs),
      func.avg(measurement.tobs)]
most_active= session.query(*stat).filter_by(station = "USC00519281").all()

print(f"The lowest temperature for {most_active[0][0]} is {most_active[0][1]}")
print(f"The highest temperature for {most_active[0][0]} is {most_active[0][2]}")
print(f"The average temperature for {most_active[0][0]} is {most_active[0][3]}")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram

prev_year = session.query(measurement.date, measurement.tobs).filter_by(station = "USC00519281").\
    filter(measurement.date >= last_year).all()

prev_year_df = pd.DataFrame(prev_year)


prev_year_df.plot(kind = "hist", bins = 12)
plt.xlabel('Temperature')
plt.title('Temperature Observations')


# Close session

In [ ]:
# Close Session
session.close()